In [2]:
# Calcula la temperatura y  
# precipitación media
# climatológica.

import os

import pandas as pd
import numpy as np

import xarray as xr

In [3]:
path_d = ["../../../resultados/CHIRTS/",
    "../../../datos/CHIRPS/"]

# Variables de CHIRTS.
vars = ["Tmax", "Tmin", "Tmean", "Pre"]

In [30]:
# Creamos un Dataset sobre el que concatenaremos
# las temperaturas medias anuales.
with xr.open_dataset( path_d[0] + vars[0] + "/mexico_" 
    + vars[0] + ".1995.nc" ) as ds_max:
    # Creamos un Dataset con las dimensiones
    # adecuadas y un solo tiempo.
    Tmean = ( ds_max.isel(time = 0)
        .rename_vars(Tmax = vars[2]) )
    Tmean["time"] = np.datetime64("1994", "ns")
    Tmean = Tmean.where(Tmean > -9000)
    Tmean = ( Tmean
        .drop_vars("time")
        .assign_coords( {"dayofyear": 0}
        ) )
    Tmax = Tmean.copy()
    Tmin = Tmean.copy()
    T = [Tmean, Tmax, Tmin]


# Calculamos la temperatura media anual.
for j in range(1995, 2015):
    # Si no existe la carpeta, la crea.
    if not os.path.exists(path_d[0] + vars[2]):
        os.mkdir(path_d[0] + vars[2])
    
    # Abrimos la temperatura máxima.
    with ( xr.open_dataset( path_d[0] + vars[0] +
        "/mexico_"  + vars[0] + "." + str(j) + ".nc" ) 
        .rename_vars(Tmax = vars[2]) ) as ds_max:
        # Establecemos np.nan como valor sin datos.
        ds_max = ds_max.where(ds_max > -9000)
        # Abrimos la temperatura mínima.
        with ( xr.open_dataset( path_d[0] + vars[1] +
            "/mexico_" + vars[1] + "." + str(j) + ".nc" ) 
            .rename_vars(Tmin = vars[2]) ) as ds_min:
            ds_min = ds_min.where(ds_min > -9000)
        
            # Calculamos la temperatura media.
            ds_mean = ( ds_max + (ds_min) ) / 2

            # Guardamos la temperatura media.
            ds_mean.to_netcdf( path_d[0] +
                vars[2] + "/mexico_" + vars[2]
                + "." + str(j) + ".nc" )

            ds_i = [ds_mean, ds_max, ds_min]

            for i in range(len(T)):
                # Retiramos los días bisiestos.
                ds_leap = ( ds_i[i].sel( 
                    time=~( (ds_i[i].time.dt.month == 2)
                    & (ds_i[i].time.dt.day == 29) ) ) )
                # Cambiamos a un año que no sea bisiesto.
                ds_leap["time"] = pd.date_range(
                    "1995-01-01", "1995-12-31",
                    freq = "1D" )
                # Obtenemos el promedio por día del año.
                ds_leap = ( ds_leap.groupby(
                    "time.dayofyear").mean("time") )
                # Concatenamos las temperaturas.
                T[i] = ( xr.concat( [T[i], ( ds_leap
                    ) ], dim = "dayofyear" ) )
                # Si no es la primera iteración,
                # obtenemos el promedio por día del año.
                if (j != 1995):
                    T[i] = ( T[i]
                        .groupby("dayofyear")
                        .mean("dayofyear") )        

for i in range(len(T)):
    # Eliminamos el primer valor de tiempo,
    # que solo sirvió como base para concatenar
    # y obtenemos el promedio temporal.
    T[i] = T[i].drop_sel(dayofyear = 0)
    # Cambiamos el índice de día a fecha.
    T[i]["dayofyear"] = pd.date_range("1995-01-01", 
        "1995-12-31", freq = "1D")
    T[i] = T[i].rename_vars(dayofyear = "time")
    T[i] = T[i].swap_dims(dayofyear = "time")
    # Guardamos el archivo diario.
    T[i].to_netcdf(path_d[0] + "mexico_"
        + vars[i] + "_daily.nc")
    # Guardamos el archivo promedio.
    T[i] = T[i].mean(dim = "time")
    T[i].to_netcdf(path_d[0] + "mexico_"
        + vars[i] + ".nc")

In [6]:
# Creamos un Dataset sobre el que concatenaremos
# la precipitación media anual.
with xr.open_dataset( path_d[1] + "/mexico_" 
    + vars[3] + ".1995.nc" ) as ds_pre:
    # Creamos un Dataset con las dimensiones
    # adecuadas y un solo tiempo.
    Pre = ds_pre.isel(time = 0)
    Pre["time"] -= 1
    Pre = Pre.where(Pre > -9000)
    Pre = ( Pre
        .drop_vars("time")
        .assign_coords( {"dayofyear": 0}
        ) )

# Calculamos la precipitación media anual.
for j in range(1995, 2015):
    
    # Abrimos la precipitación.
    with ( xr.open_dataset( path_d[1] + "/mexico_" 
        + vars[3] + "." + str(j) + ".nc" ) ) as ds_pre:
        # Establecemos np.nan como valor sin datos.
        ds_pre = ds_pre.where(ds_pre > -9000)

        # Retiramos los días bisiestos.
        ds_leap = ( ds_pre.sel( 
            time=~( (ds_pre.time.dt.month == 2)
            & (ds_pre.time.dt.day == 29) ) ) )
        # Cambiamos a un año que no sea bisiesto.
        ds_leap["time"] = pd.date_range(
            "1995-01-01", "1995-12-31", freq = "1D")
        # Obtenemos el promedio por día del año.
        ds_leap = ( ds_leap.groupby(
            "time.dayofyear").mean("time") )
        # Concatenamos las precipitaciones.
        Pre = ( xr.concat( [Pre, ( ds_leap
            ) ], dim = "dayofyear" ) )
        # Si no es la primera iteración,
        # obtenemos el promedio por día del año.
        if (j != 1995):
            Pre = ( Pre
                .groupby("dayofyear")
                .mean("dayofyear") )        

# Eliminamos el primer valor de tiempo,
# que solo sirvió como base para concatenar.
Pre = Pre.drop_sel(dayofyear = 0)
# Cambiamos el índice de día a fecha.
Pre["dayofyear"] = pd.date_range("1995-01-01", 
        "1995-12-31", freq = "1D")
Pre = Pre.rename_vars(dayofyear = "time")
Pre = Pre.swap_dims(dayofyear = "time")
# Guardamos el archivo diario.
Pre.to_netcdf(path_d[1] + "mexico_"
    + vars[3] + "_daily.nc")
# Guardamos el archivo promedio.
Pre = Pre.mean("time")
Pre.to_netcdf(path_d[1] + "mexico_"
    + vars[3] + ".nc")